In [18]:
import pandas as pd
import numpy as np

In [19]:

# #read from flat excel waste per category percentage
# df1_waste_cate_per = pd.read_excel('../source/waste_cate_per.xlsx', 'Sheet 1') 
# df2_waste_cate_per = pd.read_excel('../source/waste_cate_per.xlsx', 'Sheet 2')

#read from flat excel waste per category capita
df1_waste_cate_cap = pd.read_excel('../source/waste_cate_capita.xlsx', 'Sheet 1') 
df2_waste_cate_cap = pd.read_excel('../source/waste_cate_capita.xlsx', 'Sheet 2')
#read from flat excel food waste 
df1_food_waste = pd.read_excel('../source/food_waste.xlsx','Sheet 1')
df2_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 2')
df3_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 3')
df4_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 4')


#read from bigquery csv
df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


#read from API csv
# df_api_12 = pd.read_csv('../source/api_data12.csv')
df_api_12_3 = pd.read_csv('../source/api_data12_3.csv')

df_top_fruit_veg_wasted = pd.read_csv('../source/top_fruit_veg_wasted.csv')

df_national_fruit = pd.read_csv('../source/national_fruit.csv')
df_country_region = pd.read_csv('../source/country_region.csv')




C:\Users\krant\AppData\Local\Temp\ipykernel_22644\3165213527.py:16: DtypeWarning: Columns (9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


In [20]:

eu_country = ['Belgium',
'Bulgaria',
'Czechia',
'Denmark',
'Germany',
'Estonia',
'Ireland',
'Greece',
'Spain',
'France',
'Croatia',
'Italy',
'Cyprus',
'Latvia',
'Lithuania',
'Luxembourg',
'Hungary',
'Malta',
'Netherlands',
'Austria',
'Poland',
'Portugal',
'Romania',
'Slovenia',
'Slovakia',
'Finland',
'Sweden',
'Iceland',
'Norway',
'Switzerland',
'United Kingdom',
'Bosnia and Herzegovina',
'Montenegro',
'North Macedonia',
'Albania',
'Serbia',
'Turkey',
'Kosovo',
]




In [21]:
# Slice rows from index 12 to 38 of df1_food_waste and reset the index, dropping the old index
df_kilo_capita = df1_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita using the values from the 9th row of df1_food_waste
df_kilo_capita = df_kilo_capita.rename(columns=df1_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita and set all values in that column to 2020
df_kilo_capita['year'] = 2020

# Slice rows from index 12 to 38 of df2_food_waste and reset the index, dropping the old index
df_kilo_capita2 = df2_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita2 using the values from the 9th row of df2_food_waste
df_kilo_capita2 = df_kilo_capita2.rename(columns=df2_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita2 and set all values in that column to 2021
df_kilo_capita2['year'] = 2021

# Concatenate df_kilo_capita and df_kilo_capita2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_kilo_capita = pd.concat([df_kilo_capita, df_kilo_capita2]).reset_index(drop=True)

# First, find the column names that are NaN
nan_columns = df_kilo_capita.columns[df_kilo_capita.columns.isna()]

# Then drop these columns
df_kilo_capita_c = df_kilo_capita.drop(columns=nan_columns)
df_kilo_capita_c


df_kilo_capita_c = df_kilo_capita_c.rename(columns={'NACE_R2 (Labels)':'country'})
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_kilo_capita_c.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_kilo_capita_c.columns]


df_kilo_capita_c['id'] = df_kilo_capita_c.apply(lambda row: f'{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_kilo_capita_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_kilo_capita_c[column_to_move]

# Drop the column from its original position
df_kilo_capita_c = df_kilo_capita_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_kilo_capita_c.insert(0, column_to_move, moved_column)
# Insert the column at the first position


# Define the mapping of old names to new names
column_mapping = {
    'primary_production_of_food_-_agriculture,_fishing_and_aquaculture': 'food_production',
    'manufacture_of_food_products_and_beverages': 'food_manufacture',
    'retail_and_other_distribution_of_food': 'food_distribution',
    'restaurants_and_food_services': 'food_services',
    'total_activities_by_households': 'households'
}

# Use the replace function to rename columns
df_kilo_capita_c.rename(columns=column_mapping, inplace=True)
#cleaned

df_kilo_capita_c

# Replace ":" with NaN
df_kilo_capita_c.replace(':', np.nan, inplace=True)

# Convert columns to integers
df_kilo_capita_c['total'] = df_kilo_capita_c['total'].astype(float)
df_kilo_capita_c['food_production'] = df_kilo_capita_c['food_production'].astype(float)
df_kilo_capita_c['food_manufacture'] = df_kilo_capita_c['food_manufacture'].astype(float)

df_kilo_capita_c['food_distribution'] = df_kilo_capita_c['food_distribution'].astype(float)
df_kilo_capita_c['food_services'] = df_kilo_capita_c['food_services'].astype(float)
df_kilo_capita_c['households'] = df_kilo_capita_c['households'].astype(float)

# df_kilo_capita_c = df_kilo_capita_c['total'].astype(float)


In [22]:
df_kilo_capita.dtypes

NACE_R2 (Labels)                                                     object
Total                                                                object
NaN                                                                  object
Primary production of food - agriculture, fishing and aquaculture    object
NaN                                                                  object
Manufacture of food products and beverages                           object
NaN                                                                  object
Retail and other distribution of food                                object
NaN                                                                  object
Restaurants and food services                                        object
NaN                                                                  object
Total activities by households                                       object
NaN                                                                  object
year        

In [23]:

#read from text file 
df_composition = pd.read_csv('../source/food_composition.csv')
print(df_composition.columns)
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_composition.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_composition.columns]
df_composition_c = df_composition.drop(columns=['sugarbeets'])
df_composition_c.dtypes

df_composition_c.describe()


Index(['Country', 'Type', 'Meat', 'Fish', 'Dairy', 'Eggs', 'Cereals', 'Fruits',
       'Vegetables', 'Potatoes', 'Sugarbeets', 'Oilcrops', 'Total'],
      dtype='object')


,meat,fish,dairy,eggs,cereals,fruits,vegetables,potatoes,oilcrops,total
count,6.00000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,58.39500,67.838333,89.851667,10.405000,56.701667,289.280000,348.735000,35.480000,12.725000,969.045000
std,84.14882,38.652710,77.323586,7.075571,78.395937,359.599126,326.955547,77.068168,23.533318,742.054399
min,0.00000,27.890000,14.900000,1.350000,2.060000,2.020000,24.050000,0.000000,0.000000,148.730000
25%,0.00000,42.827500,35.995000,4.455000,17.062500,30.670000,87.530000,0.000000,0.180000,302.080000
50%,10.06500,62.260000,66.395000,14.005000,31.850000,117.355000,301.630000,4.725000,0.965000,983.605000
75%,112.69500,76.892500,127.147500,14.675000,44.972500,543.745000,510.982500,10.590000,11.695000,1603.112500
max,186.69000,136.500000,218.160000,16.900000,212.970000,811.300000,868.170000,192.460000,59.230000,1813.810000


In [24]:


#DROP EMPTY COLUMN


df_bq_keywords['geoareaname'] = df_bq_keywords['geoareaname'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")

df_bq_keywords_eu = df_bq_keywords[df_bq_keywords['geoareaname'].isin(eu_country)].reset_index(drop=True)

df_bq_keywords_eu = df_bq_keywords_eu.drop(columns=['cities','education_level','freq','hazard_type','ihr_capacity','level_status','location','migratory_status','mode_of_transportation',
'name_of_international_institution','name_of_non_communicable_disease','sex','tariff_regime_status','type_of_mobile_technology','type_of_occupation','type_of_skill','type_of_speed','goal','target','seriescode','geoareacode','nature','age','type_of_product','bounds'])
df_bq_keywords_eu['id'] = df_bq_keywords_eu.apply(lambda row: f'{row["geoareaname"].lower()}{row["timeperiod"]}', axis=1)
# Assuming 'df_bq_keywords_eu' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_bq_keywords_eu[column_to_move]

# Drop the column from its original position
df_bq_keywords_eu = df_bq_keywords_eu.drop(columns=[column_to_move])

# Insert the column at the first position
df_bq_keywords_eu.insert(0, column_to_move, moved_column)
df_bq_keywords_eu = df_bq_keywords_eu.rename(columns={'geoareaname':'country'})
df_bq_keywords_eu = df_bq_keywords_eu.rename(columns={'timeperiod':'year'})



#droping target which is not related
df_bq_keywords_eu['seriesdescription'].value_counts()
rows_to_keep = [
    "Prevalence of severe food insecurity in the adult population (%)",
    "Prevalence of moderate or severe food insecurity in the adult population (%)",
    "Adult population in severe food insecurity (thousands of people)",
    "Adult population in moderate or severe food insecurity (thousands of people)"
]

# Filter DataFrame to keep only specified rows
df_bq_food_target = df_bq_keywords_eu[df_bq_keywords_eu['seriesdescription'].isin(rows_to_keep)]


#claned
df_bq_food_target['seriesdescription'].value_counts()

# Filter DataFrame for each condition and store in separate DataFrames
df_severe_percent = df_bq_keywords_eu[df_bq_keywords_eu['seriesdescription'] == "Prevalence of severe food insecurity in the adult population (%)"].copy()
df_moderate_food_percent = df_bq_keywords_eu[df_bq_keywords_eu['seriesdescription'] == "Prevalence of moderate or severe food insecurity in the adult population (%)"].copy()
df_severe_population = df_bq_keywords_eu[df_bq_keywords_eu['seriesdescription'] == "Adult population in severe food insecurity (thousands of people)"].copy()
df_moderate_population = df_bq_keywords_eu[df_bq_keywords_eu['seriesdescription'] == "Adult population in moderate or severe food insecurity (thousands of people)"].copy()



In [25]:

df_api_12_3['goal'] = df_api_12_3['goal'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['target'] = df_api_12_3['target'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['indicator'] = df_api_12_3['indicator'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['attributes'] = df_api_12_3['attributes'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['dimensions'] = df_api_12_3['dimensions'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['footnotes'] = df_api_12_3['footnotes'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_12_3_eu = df_api_12_3[df_api_12_3['geoAreaName'].isin(eu_country)].reset_index(drop=True)
df_12_3_eu.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_12_3_eu.columns]

#DROP EMPTY COLUMN
df_12_3_eu = df_12_3_eu.drop(columns=['time_detail','timecoverage','upperbound','lowerbound','baseperiod','geoinfourl','goal','target','series','geoareacode','valuetype','footnotes'])



df_12_3_eu['timeperiodstart'] = df_12_3_eu['timeperiodstart'].astype(int)

df_12_3_eu['geoareaname'] = df_12_3_eu['geoareaname'].replace("United Kingdom of Great Britain and Northern Ireland", "United Kingdom")

df_12_3_eu['geoareaname'] = df_12_3_eu['geoareaname'].replace("Netherlands (Kingdom of the)", "Netherlands")



df_12_3_eu['id'] = df_12_3_eu.apply(lambda row: f'{row["geoareaname"].lower()}{row["timeperiodstart"]}', axis=1)
# Assuming 'df_12_3_eu' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_12_3_eu[column_to_move]

# Drop the column from its original position
df_12_3_eu = df_12_3_eu.drop(columns=[column_to_move])

# Insert the column at the first position
df_12_3_eu.insert(0, column_to_move, moved_column)

df_12_3_eu = df_12_3_eu.rename(columns={'geoareaname':'country'})
df_12_3_eu = df_12_3_eu.rename(columns={'timeperiodstart':'year'})



df_12_3_eu.dtypes

id                    object
indicator             object
seriesdescription     object
seriescount            int64
country               object
year                   int32
value                float64
source                object
attributes            object
dimensions            object
dtype: object

In [26]:
#clean

df_top_fruit_veg_wasted

,top_fruit_veg_wasted
0,Salad leaves (bagged)
1,Bananas
2,Lettuce (whole)
3,Cucumber
4,Tomatoes
5,Carrots
6,Mushrooms
7,Potato
8,Grapes
9,Strawberries


In [27]:
# 
df_national_fruit_o = pd.read_csv('../source/national_fruit.csv')
# df_national_fruit.tail(30)
df_national_fruit = df_national_fruit_o[df_national_fruit_o['country'].isin(eu_country)].reset_index(drop=True)

In [28]:
dfs_to_save = {

    'df_kilo_cap': df_kilo_capita_c,
    'df_composition': df_composition_c,
    'df_12_3_eu': df_12_3_eu,
    'df_top_wasted': df_top_fruit_veg_wasted,
    'df_national_fruit': df_national_fruit,
    'df_severe_percent': df_severe_percent,
    'df_moderate_percent': df_moderate_food_percent,
    'df_severe_population': df_severe_population,
    'df_moderate_population': df_moderate_population
}

# Save each DataFrame with its name as the CSV file name
for df_name, df in dfs_to_save.items():
    df.to_csv(f"../cleaned/{df_name}.csv", index=False)


In [29]:
original_df_shapes = {}
for df_name in ['df_kilo_capita', 'df_composition', 'df_api_12_3', 'df_national_fruit_o', 'df_bq_keywords_eu']:
    df = globals()[df_name]  # Access DataFrame by its name
    original_df_shapes[df_name] = df.shape

print("Shapes of original DataFrames:")
for df_name, shape in original_df_shapes.items():
    print(f"{df_name}: {shape}")

# For clean_df
clean_df_shapes = {}
for df_name in ['df_kilo_capita_c', 'df_composition_c', 'df_12_3_eu', 'df_country_region', 'df_national_fruit', 'df_severe_percent', 'df_moderate_food_percent', 'df_severe_population', 'df_moderate_population']:
    df = globals()[df_name]  # Access DataFrame by its name
    clean_df_shapes[df_name] = df.shape

print("\nShapes of clean DataFrames:")
for df_name, shape in clean_df_shapes.items():
    print(f"{df_name}: {shape}")

Shapes of original DataFrames:
df_kilo_capita: (54, 14)
df_composition: (6, 13)
df_api_12_3: (4744, 21)
df_national_fruit_o: (70, 2)
df_bq_keywords_eu: (62679, 10)

Shapes of clean DataFrames:
df_kilo_capita_c: (54, 9)
df_composition_c: (6, 12)
df_12_3_eu: (927, 10)
df_country_region: (49, 2)
df_national_fruit: (22, 2)
df_severe_percent: (75, 10)
df_moderate_food_percent: (75, 10)
df_severe_population: (75, 10)
df_moderate_population: (75, 10)


In [30]:
#before cleaning
original_df = df_kilo_capita,df_composition,df_api_12_3,df_national_fruit_o,df_bq_keywords_eu,
clean_df = df_kilo_capita_c,df_composition_c,df_12_3_eu,df_country_region,df_national_fruit,df_severe_percent,df_moderate_food_percent,df_severe_population,df_moderate_population,

In [31]:
# # For original_df
# for df in [df_waste_cap, df_kilo_capita, df_tonne, df_composition, df_bq_keywords_eu, df_api_12_3, df_country_region, df_national_fruit_o]:
#     print(df.columns)

# # For clean_df
# for df in [df_waste_cap_c, df_kilo_capita_c, df_tonne_c, df_composition_c, df_severe_percent, df_moderate_food_percent, df_severe_population, df_moderate_population, df_12_3_eu, df_country_region, df_national_fruit]:
#     print(df.columns)